In [14]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from http import HTTPStatus
import json 
import dashscope
from openai import OpenAI
from typing import List, Dict, Tuple, Any

In [15]:
load_dotenv()

True

In [16]:
XAI_API_KEY = os.getenv("XAI_API_KEY", "")
XAI_BASE_URL = os.getenv("XAI_BASE_URL", "")
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "")
QDRANT_URL = os.getenv("QDRANT_URL", "")

In [17]:
client = OpenAI(
    base_url=XAI_BASE_URL,
    api_key=XAI_API_KEY
)

In [18]:
def get_current_temperature(location: str, unit: str = "fahrenheit"):
    temperature: int
    if unit.lower() == "fahrenheit":
        temperature = 59
    elif unit.lower() == "celsius":
        temperature = 15
    else:
        raise ValueError("unit must be one of fahrenheit or celsius")
    return {"location": location, "temperature": temperature, "unit": "fahrenheit"}


def get_current_ceiling(location: str):
    return {
        "location": location,
        "ceiling": 15000,
        "ceiling_type": "broken",
        "unit": "ft",
    }

tools_map = {
    "get_current_temperature": get_current_temperature,
    "get_current_ceiling": get_current_ceiling,
}


In [26]:
qdrant_client = QdrantClient(url=QDRANT_URL)


dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

def embed_with_str(query: str):
    resp = dashscope.TextEmbedding.call(
        model=dashscope.TextEmbedding.Models.text_embedding_v3,
        api_key=DASHSCOPE_API_KEY,
        input=query)
    if resp.status_code == HTTPStatus.OK:
        return resp.output['embeddings'][0]['embedding']
    else:
        print(resp)

def search_knowledge_base(query: str) -> Tuple[List[Dict[str, Any]], str]:
    """
    Search the knowledge base for relevant information.

    Args:
        query: str -> query used to retrieve the relevant information.
    Returns:
        Tuple containing:
        1. List of the formatted results with title, content preview, and source link
        2. String joining all relevant information from results
    """

    embed = embed_with_str(query)
    results = qdrant_client.query_points(
        collection_name="knowledge_base_collection",
        query=embed,
        with_payload=True,
        limit=3,
        score_threshold=0.6
    )
    formatted_results = []
    combined_text_parts = []
    for result in results.points:
        # Format dictionary result
        content_preview = result.payload["content"][:200] + "..." if len(result.payload["content"]) > 200 else result.payload["content"]
        formatted_results.append({
            "title": result.payload["title"],
            "content_preview": content_preview,
            "source_link": result.payload["source_link"],
            "relevance_score": round(result.score, 3)
        })

        # Add to combined text
        combined_text_parts.append(
            f"Title: {result.payload['title']}\n"
            f"Content: {result.payload['content']}\n"
            f"Source: {result.payload['source_link']}\n"
        )

    combined_text = "\n".join(combined_text_parts)
    return formatted_results, combined_text

def search_doctors(query: str) -> Tuple[List[Dict[str, Any]], str]:
    """
    Search for doctors based on query.
    Returns:
        Tuple containing:
        1. List of formatted results with doctor information
        2. String joining all relevant information from results
    """
    embed = embed_with_str(query)
    results = qdrant_client.query_points(
        collection_name="doctor_collection",
        query=embed,
        with_payload=True,
        limit=3,
        score_threshold=0.5,
    )

    formatted_results = []
    combined_text_parts = []

    for result in results.points:
        # Format dictionary result
        formatted_results.append({
            "doctor_name": result.payload["doctor_name"],
            "specialization": result.payload["doctor_field"],
            "description": result.payload["doctor_description"],
            "availability_status": "Available" if result.payload["availability"] else "Not Available",
            "appointment_link": result.payload["appointment_link"],
            "relevance_score": round(result.score, 3)
        })

        # Add to combined text
        combined_text_parts.append(
            f"Specialization: {result.payload['doctor_field']}\n"
            f"Description: {result.payload['doctor_description']}\n"
        )

    combined_text = "\n".join(combined_text_parts)
    return formatted_results, combined_text

In [27]:
tools_definition = [
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Retrieve health information from the knowledge base given the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user query, e.g. what is diabetes ?"
                    },
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_doctors",
            "description": "Get the suitalbe doctors based on the user's query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user query, e.g. recommend me a doctor for diabetes"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

tools_map = {
    "search_knowledge_base": search_knowledge_base,
    "search_doctors": search_doctors,
}


In [34]:
global messages
messages = []

In [35]:
messages = [{"role": "user", "content": "Hi, what is acne ?"}]

In [40]:
response = client.chat.completions.create(
    model="grok-2-1212",
    messages=messages,
    tools=tools_definition,
    tool_choice="auto",
    stream=True
)

fn_call_in_progress = False
fn_results = []

for chunk in response:

    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)


    if chunk.choices[0].delta.tool_calls:
        print("\n")
        
        for tool_call in chunk.choices[0].delta.tool_calls:

            fn_name = tool_call.function.name
            fn_args = json.loads(chunk.choices[0].delta.tool_calls[0].function.arguments)


            result = tools_map[fn_name](**fn_args)

            messages.append(
                {
                    "role": "tool",
                    "content": json.dumps(result),
                    "tool_call_id": tool_call.id  # tool_call.id supplied in Grok's response
                }
            )

if messages[-1]['role'] == 'tool':
    response = client.chat.completions.create(
        model="grok-2-1212",
        messages=messages,
        tools=tools_definition,
        tool_choice="auto",
        stream=True
    )

    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)  

I am searching for a suitable doctor for your acne condition.

I apologize, but the doctors I found do not specialize in acne treatment. Would you like me to search again for dermatologists or other specialists who might be more suitable for your needs?

In [39]:
messages.append({
    'role': 'user',
    'content': 'Suggest me a doctor for acne'
})